In [ ]:
from data_functions import his_usage_team
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

{ 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
{'Chris Paul': 'SAS',"De'Aaron Fox": 'SAS', "Devin Vassell": 'SAS',"Harrison Barnes": 'SAS'}

player_names = {"Payton Pritchard": 'BOS'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


player_df, current_defense_stat = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)



for player, df in player_df.items():
    print(player)


    from sklearn.linear_model import Lasso
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    import numpy as np

    df_X = df.drop(columns=['PTS','Date','Matchup','Team','Home/Away_game','W/L', 'Away', 'season', 'TEAM', 'season_defense'])



    scaler = StandardScaler()
    X = scaler.fit_transform(df_X)

    y = df['PTS']  # Replace with your actual target


    # Define the grid search parameters for Lasso (L1 regularization)
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1, 10]  # Different levels of regularization strength
    }

    # Perform grid search
    grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='r2')
    grid_search.fit(X, y)

    # Print best parameters
    print("Best alpha:", grid_search.best_params_)

    X = pd.DataFrame(X, columns=df_X.columns)  # Convert back to DataFrame


    # Get the best alpha
    best_alpha = grid_search.best_params_['alpha']

    # Fit Lasso with the best alpha
    best_lasso = Lasso(alpha=best_alpha)
    best_lasso.fit(X, y)


    # Get selected (non-zero) feature indices
    selected_features = X.columns[best_lasso.coef_ != 0]
    print("Selected features:", selected_features)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # display(y_test)

    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp

    X_train = train_data[selected_features]
    y_train = train_data['FGM']
    X_test = test_data[selected_features]
    y_test = test_data['FGM']



    # Reduce X to selected features only
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]


    # Retrain Lasso only on selected features
    final_lasso = Lasso(alpha=best_alpha)
    final_lasso.fit(X_train_selected, y_train)

    # Predict on the test set using the reduced feature set
    y_pred = final_lasso.predict(X_test_selected)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Final RMSE using selected features:", rmse)


In [6]:
from data_functions import his_usage_team

player_names = {
    "Jayson Tatum": "BOS",
    # # "Nikola Jokic": "DEN",
    # "Jamal Murray": "DEN",
    # "Jaylen Brown": "BOS",
    # "Derrick White": "BOS",
    # "Payton Pritchard": "BOS",
    # "Michael Porter Jr.": "DEN",
    # "Russell Westbrook": "DEN",
    # "Christian Braun": "DEN",
    # "Al Horford": "BOS",
    # # "Julian Strawther": "DEN",
    # "Sam Hauser": "BOS",
    # "Zeke Nnaji": "DEN",
    # "Luke Kornet": "BOS"
}
date_list = ["2023-24","2024-25"]
# usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
player_df, current_defense_stat = his_usage_team(player_names, date_list, stats_path, player_base_path, defense_base_path)

for player, df in player_df.items():
    display(player)
    display(df.head(5))

'Jayson Tatum'

,Date,Matchup,Team,Away,Home/Away_game,W/L,MIN_x,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,season,RANK,TEAM,GP,W,L,MIN_y,OffRtg,DefRtg,NetRtg,AST%,AST/TO,ASTRatio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE,POSS,season_defense,RANK_usg,TEAM_usg,AGE_usg,GP_usg,W_usg,L_usg,MIN_usg,USG%_usg,%FGM_usg,%FGA_usg,%3PM_usg,%3PA_usg,%FTM_usg,%FTA_usg,%OREB_usg,%DREB_usg,%REB_usg,%AST_usg,%TOV_usg,%STL_usg,%BLK_usg,%BLKA_usg,%PF_usg,%PFD_usg,%PTS_usg,season,TEAM_catch_shoot,GP_catch_shoot,MIN_catch_shoot,PTS_catch_shoot,FGM_catch_shoot,FGA_catch_shoot,FG%_catch_shoot,3PM_catch_shoot,3PA_catch_shoot,3P%_catch_shoot,eFG%_catch_shoot,season,TEAM_drive,GP_drive,W_drive,L_drive,MIN_drive,DRIVES_drive,FGM_drive,FGA_drive,FG%_drive,FTM_drive,FTA_drive,FT%_drive,PTS_drive,PTS%_drive,PASS_drive,PASS%_drive,AST_drive,AST%_drive,TO_drive,TOV%_drive,PF_drive,PF%_drive,season,TEAM_elbow,GP_elbow,W_elbow,L_elbow,MIN_elbow,Touches_elbow,ElbowTouches_elbow,FGM_elbow,FGA_elbow,FG%_elbow,FTM_elbow,FTA_elbow,FT%_elbow,PTS_elbow,PTS%_elbow,PASS_elbow,PASS%_elbow,AST_elbow,AST%_elbow,TO_elbow,TOV%_elbow,PF_elbow,PF%_elbow,season,TEAM_paint,GP_paint,W_paint,L_paint,MIN_paint,Touches_paint,PaintTouches_paint,FGM_paint,FGA_paint,FG%_paint,FTM_paint,FTA_paint,FT%_paint,PTS_paint,PTS%_paint,Pass_paint,Pass%_paint,AST_paint,AST%_paint,TO_paint,TOV%_paint,PF_paint,PF%_paint,season,TEAM_passing,GP_passing,W_passing,L_passing,MIN_passing,PassesMade_passing,PassesReceived_passing,AST_passing,SecondaryAST_passing,PotentialAST_passing,ASTPTSCreated_passing,ASTAdj_passing,Assist_to_Pass_passing,Assist_to_Pass_Percentage_Adj_passing,season,TEAM_pullup,GP_pullup,W_pullup,L_pullup,MIN_pullup,PTS_pullup,FGM_pullup,FGA_pullup,FG%_pullup,3PM_pullup,3PA_pullup,3P%_pullup,eFG%_pullup,season,TEAM_shot_efg,GP_shot_efg,W_shot_efg,L_shot_efg,MIN_shot_efg,PTS_shot_efg,DrivePTS_shot_efg,DriveFG%_shot_efg,C&SPTS_shot_efg,C&SFG%_shot_efg,Pull UpPTS_shot_efg,Pull UpFG%_shot_efg,PaintTouch PTS_shot_efg,PaintTouch FG%_shot_efg,PostTouch PTS_shot_efg,PostTouch FG%_shot_efg,ElbowTouch PTS_shot_efg,ElbowTouch FG%_shot_efg,eFG%_shot_efg,season,Team_touches,GP_touches,W_touches,L_touches,MIN_touches,PTS_touches,TOUCHES_touches,Front CTTouches_touches,Time OfPoss_touches,Avg Sec PerTouch_touches,Avg Drib PerTouch_touches,PTS PerTouch_touches,ElbowTouches_touches,PostUps_touches,PaintTouches_touches,PTS PerElbow Touch_touches,PTS PerPost Touch_touches,PTS PerPaint Touch_touches,season,TEAM_post_ups,GP_post_ups,W_post_ups,L_post_ups,MIN_post_ups,Touches_post_ups,PostUps_post_ups,FGM_post_ups,FGA_post_ups,FG%_post_ups,FTM_post_ups,FTA_post_ups,FT%_post_ups,PTS_post_ups,PTS%_post_ups,PASS_post_ups,PASS%_post_ups,AST_post_ups,AST%_post_ups,TO_post_ups,TOV%_post_ups,PF_post_ups,PF%_post_ups,season,team_pace,team_offrtg,team_poss,Date_in_Seconds,home_away
0,2023-12-17,BOS vs. ORL,BOS,ORL,Home,W,30.52,23,8,21,38.1,4,6,66.7,3,4,75.0,3,5,8,3,0,1,1,3,9,2023-24,12,ORL,82,47,35,3956.0,112.9,110.8,2.2,61.0,1.68,18.1,29.7,73.7,51.5,15.0,54.1,57.7,97.37,51.3,8022,2023-24,19,BOS,26,74,57,17,2645.0,29.6,28.1,29.1,25.4,26.1,44.9,44.3,12.4,27.5,24.2,25.0,29.7,20.1,12.3,33.9,16.7,35.4,30.0,2023-24,BOS,74,35.7,3.1,1.1,2.5,42.6,1.0,2.2,43.0,61.4,2023-24,BOS,73,56,17,35.7,9.1,2.8,5.0,57.3,1.9,2.3,84.5,7.9,86.4,2.3,25.2,0.7,7.7,0.4,4.8,1.2,12.8,2023-24,BOS,73,56,17,35.7,75.8,1.2,0.4,0.6,62.2,0.3,0.3,91.7,1.1,92.9,0.3,28.2,0.1,25.0,0.0,2.4,0.2,14.1,2023-24,BOS,73,56,17,35.7,75.8,1.6,0.7,1.2,63.1,0.3,0.4,76.9,1.7,109.5,0.2,11.2,0.0,2.6,0.1,3.4,0.2,11.2,2023-24,BOS,73,56,17,35.7,49.3,49.7,4.9,1.0,8.9,13.0,6.2,9.9,12.5,2023-24,BOS,74,57,17,35.7,8.3,3.2,8.7,36.3,2.0,5.6,34.9,47.5,2023-24,BOS,73,56,17,35.7,26.8,7.9,57.3,3.6,42.8,8.4,36.3,1.7,63.1,1.4,47.2,1.1,62.2,55.3,2023-24,BOS,73,56,17,35.7,26.8,75.8,38.7,4.8,3.83,2.76,0.354,1.2,1.8,1.6,0.929,0.773,1.095,2023-24,BOS,73,56,17,35.7,75.8,1.8,0.5,1.0,47.2,0.4,0.5,76.5,1.4,77.3,0.4,20.3,0.0,0.8,0.1,5.5,0.2,13.3,2023-24,97.98,122.2,8090,1702771200

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import pandas as pd
from data_functions import his_usage_team

def select_features(player_names, date_list, usage_path, player_base_path, defense_base_path, target):
    player_df, _ = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    
    selected_features_dict = {}
    
    max_features_player = None
    max_features = 0

    for player, df in player_df.items():
        df_X = df.drop(columns=[target,'Date','Matchup','Team','Home/Away_game','W/L', 'Away', 'season', 'TEAM', 'season_defense',
                                ])
        
        scaler = StandardScaler()
        X = scaler.fit_transform(df_X)
        y = df[target]  # Target variable
        # Grid search parameters for Lasso
        
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='r2')
        grid_search.fit(X, y)
        
        # Get the best alpha and fit Lasso
        best_alpha = grid_search.best_params_['alpha']
        best_lasso = Lasso(alpha=best_alpha)
        best_lasso.fit(X, y)
        
        # Select non-zero coefficient features
        X = pd.DataFrame(X, columns=df_X.columns)
        selected_features = X.columns[best_lasso.coef_ != 0].tolist()
        
        # Store selected features
        selected_features_dict[player] = selected_features
        
        # Track the player with the most features
        if len(selected_features) > max_features:
            max_features = len(selected_features)
            max_features_player = player

    # If a player has no selected features, assign the features of the player with the most features
    for player in selected_features_dict:
        if not selected_features_dict[player]:  # If empty
            selected_features_dict[player] = selected_features_dict.get(max_features_player, [])

    return selected_features_dict


player_names = {
    "Devin Booker": "PHX",
    "Anthony Edwards": "MIN",
    "Kevin Durant": "PHX",
    "Naz Reid": "MIN",
    "Julius Randle": "MIN",
    "Bradley Beal": "PHX",
    "Bol Bol": "PHX",
    "Donte DiVincenzo": "MIN",
    "Jaden McDaniels": "MIN",
    "Nick Richards": "PHX",
    "Mike Conley": "MIN"
}
date_list = ["2022-23","2023-24","2024-25"]
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

feature_dic = select_features(player_names, date_list, stats_path, player_base_path, defense_base_path,'FGA')

for player, features in feature_dic.items():
    # print(player)
    features = feature_dic[player] 
    print(player,':',features)


ValueError: could not convert string to float: 'PHX'

In [ ]:
from data_functions import his_usage_team
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def select_features(player_names, date_list, usage_path, player_base_path, defense_base_path, target):
    player_df, _ = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    
    selected_features_dict = {}
    
    for player, df in player_df.items():
        df_X = df.drop(columns=[target,'Date','Matchup','Team','Home/Away_game','W/L', 'Away', 'season', 'TEAM', 'season_defense'])
        
        scaler = StandardScaler()
        X = scaler.fit_transform(df_X)
        y = df[target]  # Target variable
        
        # Grid search parameters for Lasso
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='r2')
        grid_search.fit(X, y)
        
        # Get the best alpha and fit Lasso
        best_alpha = grid_search.best_params_['alpha']
        best_lasso = Lasso(alpha=best_alpha)
        best_lasso.fit(X, y)
        
        # Select non-zero coefficient features
        X = pd.DataFrame(X, columns=df_X.columns)
        selected_features = X.columns[best_lasso.coef_ != 0].tolist()
        selected_features_dict[player] = selected_features
        
    return selected_features_dict


player_names = {
    "Devin Booker": "PHX",
    "Anthony Edwards": "MIN",
    "Kevin Durant": "PHX",
    "Naz Reid": "MIN",
    "Julius Randle": "MIN",
    "Bradley Beal": "PHX",
    "Bol Bol": "PHX",
    "Donte DiVincenzo": "MIN",
    "Jaden McDaniels": "MIN",
    "Nick Richards": "PHX",
    "Mike Conley": "MIN"
}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


feature_dic = select_features(player_names, date_list, usage_path, player_base_path, defense_base_path,'AST')

for player, features in feature_dic.items():
    # print(player)
    features = feature_dic[player] 
    print(player,':',features)

In [19]:
import pandas as pd

# Sample data (replace with your actual DataFrame)
data = {
    'player_id': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 
                  'B', 'B', 'B', 'B', 'B', 'B', 'B'],
    'PTS': [10, 5, 35, 6, 0, 1, 56, 18, 20, 22, 19, 21, 20, 22],
    'REB': [4, 7, 6, 5, 2, 3, 8, 6, 5, 8, 7, 5, 6, 7],
    'AST': [2, 3, 7, 4, 1, 2, 5, 4, 3, 6, 2, 5, 4, 3]
}

df = pd.DataFrame(data)

# Group by player_id and calculate std and mean for each player's PTS, REB, and AST
player_variability = df.groupby('player_id')[['PTS', 'REB', 'AST']].agg(['std', 'mean'])

# Flatten the multi-level columns (from the aggregation)
player_variability.columns = [f"{col[0]}_{col[1]}" for col in player_variability.columns]

# Calculate Coefficient of Variation (CV)
player_variability["CV_PTS"] = player_variability["PTS_std"] / player_variability["PTS_mean"]
player_variability["CV_REB"] = player_variability["REB_std"] / player_variability["REB_mean"]
player_variability["CV_AST"] = player_variability["AST_std"] / player_variability["AST_mean"]

# Sort by CV_PTS (assuming you want to sort based on the points variability)
player_variability = player_variability.sort_values(by="CV_PTS", ascending=False)

print(player_variability)


             PTS_std   PTS_mean   REB_std  REB_mean   AST_std  AST_mean    CV_PTS    CV_REB    CV_AST
player_id                                                                                            
A          21.208713  16.142857  2.160247  5.000000  2.070197  3.428571  1.313814  0.432049  0.603807
B           1.496026  20.285714  1.112697  6.285714  1.345185  3.857143  0.073748  0.177020  0.348752


In [37]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# features for shai
# 'MIN_x', 'FGM', 'FG%', '3PA', 'OREB', 'REB', 'RANK'

from feature_function import his_usage_team, select_features
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display
from scipy.stats import linregress



# player_names = {"Payton Pritchard": 'BOS'}

player_names = {
    "LeBron James": "LAL",
    "Luka Doncic": "LAL",
    "Jayson Tatum": "BOS",
    "Jaylen Brown": "BOS",
    "Kristaps Porzingis": "BOS",
    "Austin Reaves": "LAL",
    "Derrick White": "BOS",
    "Payton Pritchard": "BOS",
    "Jrue Holiday": "BOS",
    "Dorian Finney-Smith": "LAL",
    # "Dalton Knecht": "LAL",
    "Gabe Vincent": "LAL",
    "Damian Lillard": "MIL",
    "Giannis Antetokounmpo": "MIL",
    "Franz Wagner": "ORL",
    "Paolo Banchero": "ORL",
    "Kyle Kuzma": "MIL",
    "Brook Lopez": "MIL",
    "Cole Anthony": "ORL",
    "Wendell Carter Jr.": "ORL",
    "Taurean Prince": "MIL",
    "Kentavious Caldwell-Pope": "ORL",
    # "Anthony Black": "ORL",
    "Jonathan Isaac": "ORL",
    "Goga Bitadze": "ORL",
    # "Tristan da Silva": "ORL"
}


date_list = ["2022-23","2023-24","2024-25"]
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def prediction(player_names: dict, date_list: list, stats_path, player_base_path, defense_base_path, schedule_base_path,selected_feature_target, prediction_target):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, stats_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import pandas

    # this is the function that will be used to select the features for the model prints out the player and the best features to use
    feature_dic = select_features(player_names, date_list, stats_path, player_base_path, defense_base_path,selected_feature_target)




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]

        # Calculate standard deviation
        std_dev_pts = df["PTS"].std()
        std_dev_reb = df["REB"].std()

        # Calculate mean
        mean_pts = df["PTS"].mean()
        mean_reb = df["REB"].mean()

        # Calculate coefficient of variation
        cv_pts = std_dev_pts / mean_pts
        cv_reb = std_dev_reb / mean_reb

        print(f"PTS - Std Dev: {std_dev_pts}, CV: {cv_pts}")
        print(f"REB - Std Dev: {std_dev_reb}, CV: {cv_reb}")



        df["Rolling_Mean_PTS"] = df["PTS"].rolling(window=30).mean()
        df["Rolling_Std_PTS"] = df["PTS"].rolling(window=30).std()
        df["Rolling_CV_PTS"] = df["Rolling_Std_PTS"] / df["Rolling_Mean_PTS"]

        df["Rolling_Mean_REB"] = df["REB"].rolling(window=30).mean()
        df["Rolling_Std_REB"] = df["REB"].rolling(window=30).std()
        df["Rolling_CV_REB"] = df["Rolling_Std_REB"] / df["Rolling_Mean_REB"]

        print(df[["Rolling_Mean_PTS", "Rolling_Std_PTS", "Rolling_CV_PTS",
                "Rolling_Mean_REB", "Rolling_Std_REB", "Rolling_CV_REB"]].tail(15))
        


        # Get last 10 games
        recent_games = df["PTS"].tail(10)

        # Fit a linear regression (x = game number, y = points)
        slope, intercept, r_value, p_value, std_err = linregress(range(len(recent_games)), recent_games)

        # If slope is negative, player is trending down
        if slope < 0:
            print("Player is trending down")
        else:
            print("Player is stable or trending up")

        


        features = feature_dic[player] 
        # print(features)
        target = prediction_target

        timestamp = int(pd.Timestamp('2025-02-26').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        # display(test_data)

        X_train = train_data[features].fillna(0)
        y_train = train_data[target].fillna(0)
        X_test = test_data[features].fillna(0)
        y_test = test_data[target].fillna(0) 

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)


        # Exclude columns
        exclude_features = ['RANK', 'OffRtg', 'W', 'L', 
            'DefRtg','NetRtg', 'AST%', 'AST/TO', 'ASTRatio',
            'OREB%', 'DREB%', 'REB%', 'TOV%' , 'eFG%', 'TS%', 'PACE',
            'POSS', 'TEAM', 'PIE']
        
        exclude_features_schedule = ['home_away', 'schedule_team', 'DATE', 'location', 'season_defense']

        # Get defensive stats for the scheduled team
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # df_defense = df_defense.merge(schedule_df, left_on='TEAM', right_on = 'schedule_team', how='outer', suffixes=('', '_DROP'))
        # df_defense = df_defense.drop(columns=[col for col in df_defense.columns if col.endswith('_DROP')])
        # df_defense= df_defense.fillna(0)


        # display(df_for_schedule)

        


        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, exclude_features]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]


        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]


        schedule_values = {feature: df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, feature].values[0] 
                for feature in exclude_features if feature in df_for_schedule.columns}

        print(schedule_values)


        # checking to see if the schedule values are being passed correctly
        
        rolling_features = [col for col in features  if col not in exclude_features]
        # display(rolling_features)

        # df = df[df['Date_in_Seconds'] >= timestamp]
        # # display(df)

        for col in rolling_features:
            df[f'{col}'] = df[col].rolling(window=20).mean().fillna(0).astype(int)

        df_last_rolling = df.iloc[[-1]][[f'{col}' for col in rolling_features]]

        df_last_rolling = df_last_rolling.reset_index(drop=True)

        for value in features:
            if value in exclude_features:
                print(f'This is the value {value}')
                df_last_rolling[value] = schedule_values.get(value)
                # df_last_rolling[value] = schedule_df[value].iloc[0]

        for value in features:
            if value in exclude_features_schedule:
                print(f'This is the value {value}')
                # df_last_rolling[value] = schedule_values.get(value)
                df_last_rolling[value] = schedule_df[value].iloc[0]        



        df_last_rolling = df_last_rolling.reindex(columns=features)

        X_future = df_last_rolling

        display(X_future)

        future_predictions = model.predict(X_future).astype('int')
        fga_prediction_results[player] = [future_predictions[0].round(1), rmse]
        df_results = pd.DataFrame.from_dict(fga_prediction_results, orient='index', columns=[target, 'RMSE'])
        # Reset index and rename it properly
        df_results.reset_index(inplace=True)
        df_results.rename(columns={'index': 'Player'}, inplace=True)
        # display(df_results)


    return   df_results

results = prediction(player_names, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'PTS','PTS')


# for player, fga_predictions in results.items():
#     print(fga_predictions)

display(results)


PTS - Std Dev: 7.702607130027635, CV: 0.2860586950988228
REB - Std Dev: 3.1705862943096865, CV: 0.40235866679056936
     Rolling_Mean_PTS  Rolling_Std_PTS  Rolling_CV_PTS  Rolling_Mean_REB  Rolling_Std_REB  Rolling_CV_REB
135         24.033333         6.950482        0.289202          7.000000         3.039964        0.434281
136         24.266667         7.133860        0.293978          7.300000         2.972866        0.407242
137         24.100000         7.028759        0.291650          7.233333         2.932380        0.405398
138         24.900000         7.648980        0.307188          7.600000         3.420022        0.450003
139         25.100000         7.540008        0.300399          7.566667         3.420862        0.452096
140         25.166667         7.465245        0.296632          7.433333         3.400642        0.457486
141         25.633333         7.039217        0.274612          7.400000         3.399797        0.459432
142         26.066667         7.5106

,FGM,3PM,FTM,FTA
0,10,2,4,5


PTS - Std Dev: 10.053438021293712, CV: 0.3172140692433047
REB - Std Dev: 3.1245047101564, CV: 0.3549303203844926
     Rolling_Mean_PTS  Rolling_Std_PTS  Rolling_CV_PTS  Rolling_Mean_REB  Rolling_Std_REB  Rolling_CV_REB
112         29.866667         7.271737        0.243473          9.200000         2.203446        0.239505
113         29.166667         7.566183        0.259412          9.233333         2.223438        0.240806
114         30.066667         7.794487        0.259240          9.266667         2.242741        0.242022
115         29.833333         7.777436        0.260696          9.200000         2.280351        0.247864
116         29.333333         8.297625        0.282874          8.966667         2.341284        0.261110
117         28.866667         8.756213        0.303333          8.766667         2.416657        0.275664
118         28.533333         9.054370        0.317326          8.600000         2.567704        0.298570
119         27.433333         8.732355 

,FGM,3PM,3PA,3P%,FTM
0,8,3,9,34,5


PTS - Std Dev: 8.11460417363993, CV: 0.29457662779428595
REB - Std Dev: 3.1604448473450093, CV: 0.36551019822802727
     Rolling_Mean_PTS  Rolling_Std_PTS  Rolling_CV_PTS  Rolling_Mean_REB  Rolling_Std_REB  Rolling_CV_REB
135         25.033333         7.425415        0.296621          9.366667         2.846454        0.303892
136         25.400000         7.233066        0.284766          9.266667         2.911728        0.314215
137         25.700000         7.442423        0.289588          9.233333         2.932380        0.317586
138         25.766667         7.398431        0.287132          9.066667         3.084257        0.340175
139         25.166667         7.353551        0.292194          8.733333         3.073057        0.351877
140         25.400000         7.712954        0.303660          8.666667         3.110974        0.358959
141         25.900000         7.702888        0.297409          8.566667         3.081461        0.359704
142         25.833333         7.6387

...

PTS - Std Dev: 4.424542527378009, CV: 0.706653554732315
REB - Std Dev: 2.537612231897506, CV: 0.5375476292759984
     Rolling_Mean_PTS  Rolling_Std_PTS  Rolling_CV_PTS  Rolling_Mean_REB  Rolling_Std_REB  Rolling_CV_REB
96           6.100000         4.830578        0.791898          4.833333         2.730206        0.564870
97           6.266667         4.947541        0.789501          5.000000         2.766798        0.553360
98           6.200000         4.964703        0.800759          5.066667         2.740920        0.540971
99           6.233333         4.938821        0.792324          4.966667         2.760351        0.555775
100          6.133333         4.994710        0.814355          4.766667         2.430884        0.509976
101          5.766667         5.028768        0.872041          4.666667         2.440087        0.522876
102          5.633333         5.129249        0.910518          4.600000         2.429701        0.528196
103          5.833333         5.337462 

,FGM,3PM,FTM
0,1,0,0


PTS - Std Dev: 4.878537568846667, CV: 0.8180656633194534
REB - Std Dev: 4.26808027471717, CV: 0.7869811542883611
     Rolling_Mean_PTS  Rolling_Std_PTS  Rolling_CV_PTS  Rolling_Mean_REB  Rolling_Std_REB  Rolling_CV_REB
122          9.733333         4.975341        0.511165          8.500000         4.329131        0.509310
123          9.866667         4.811863        0.487689          8.833333         4.355601        0.493087
124          9.766667         4.688087        0.480009          8.733333         4.362326        0.499503
125          9.600000         4.803734        0.500389          8.766667         4.312638        0.491936
126          9.433333         4.980537        0.527972          8.566667         4.149643        0.484394
127          9.166667         5.265917        0.574464          8.633333         4.038464        0.467776
128          9.000000         5.445656        0.605073          8.733333         3.956081        0.452986
129          8.833333         5.458832 

,FGM,3PM,FTM,FTA
0,2,0,0,1


,Player,PTS,RMSE
0,LeBron James,26,5.024296e-15
1,Luka Doncic,23,4.493867e-15
2,Jayson Tatum,25,7.536444e-15
3,Jaylen Brown,19,5.801557e-15
4,Kristaps Porzingis,16,3.552714e-15
5,Austin Reaves,17,8.940800e-15
6,Derrick White,14,1.103631e-14
7,Payton Pritchard,13,3.177644e-15
8,Jrue Holiday,7,2.664535e-15
9,Dorian Finney-Smith,4,9.930137e-16


In [1]:
# features for shai
# 'MIN_x', 'FGM', 'FG%', '3PA', 'OREB', 'REB', 'RANK'

from feature_function import his_usage_team
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display


player_names ={"Jaylen Brown": 'BOS'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import pandas




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]


        features = selected_features 
        print(features)
        target = 'FGA'

        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        display(test_data)

        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]

        # # Initialize Scaler
        # scaler = StandardScaler()
        # # Transform Data
        # scaled_data_x = scaler.fit_transform(X_train)
        # scaled_data_y = scaler.fit_transform(y_train.values.reshape(-1, 1))

        # X_train = pd.DataFrame(scaled_data_x, columns=X_train.columns)     
        # y_train = pd.DataFrame(scaled_data_y, columns=['features'])   

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)


        # Exclude columns
        exclude_features = ['RANK', 'OffRtg', 'W', 'L', 
            'DefRtg','NetRtg', 'AST%', 'AST/TO', 'ASTRatio',
            'OREB%', 'DREB%', 'REB%', 'TOV%' , 'eFG%', 'TS%', 'PACE',
            'POSS', 'TEAM', 'home_away', 'PIE']

        # Get defensive stats for the scheduled team
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        df_defense = df_defense.merge(schedule_df, left_on='TEAM', right_on = 'schedule_team', how='outer')

        # display(df_for_schedule)

        


        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, exclude_features]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]
        # could refactor to use squeeze but simpler to read this way
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]


        schedule_values = {feature: df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, feature].values[0] 
                   for feature in exclude_features if feature in df_for_schedule.columns}

        print(schedule_values)


        # checking to see if 
        
        rolling_features = [col for col in selected_features  if col not in exclude_features]

        for col in rolling_features:
            df[f'{col}'] = df[col].rolling(window=10).mean()

        df_last_rolling = df.iloc[[-1]][[f'{col}' for col in rolling_features]]

        df_last_rolling = df_last_rolling.reset_index(drop=True)

        for value in selected_features:
            if value in exclude_features:
                print(f'This is the value {value}')
                df_last_rolling[value] = schedule_values.get(value)



        df_last_rolling = df_last_rolling.reindex(columns=selected_features)


        X_future = df_last_rolling

        display(X_future)

        future_predictions = model.predict(X_future)
        fga_prediction_results[player] = player , future_predictions[0].round(1)

    return fga_prediction_results

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)


NameError: name 'selected_features' is not defined

In [ ]:
from feature_function import fga_prediction


SAS = {'Chris Paul': 'SAS',"De'Aaron Fox": 'SAS', "Devin Vassell": 'SAS',"Harrison Barnes": 'SAS'}
OKC = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}

player_names = SAS
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)


In [ ]:
def build_data_path(base_path, **Kwargs):
    # Replace placeholders with actual values
    for key, value in Kwargs.items():
        base_path=base_path.replace(f"{{{key}}}", str(value))
    return base_path

base_path = "/data/{year}/{month}/{day}/file.txt"
kwargs = {'year': 2025, 'month': '02', 'day': '10'}
year = 2025
build_data_path(base_path, year=year)

In [ ]:
import pandas as pd

schedule_df = pd.read_csv("D:/nba_scheduled_csv/schedule_csv_2025/OKC_schedule_content.csv")

schedule_df

In [ ]:
def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list,usage_path,player_base_path, defense_base_path)

    fga_prediction_results = {}

    for player, team in player_names.items():

        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # print(schedule_df)

    for player, df in fga_prediction_data.items():

        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.metrics import mean_squared_error
        from sklearn.model_selection import cross_val_score
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        import numpy as np



        features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
        target = 'FGA'


        # Continue with your existing operations
        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
        test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]


        model = LinearRegression()
        model.fit(X_train,y_train)

        y_pred = model.predict(X_test)

        ###### predicting Minutes ##########
        alpha = 0.2

        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()

        last_actual = df['MIN_x'].iloc[-1]  # Last known FGA
        last_smoothed = df['EWMA_MIN'].iloc[-1]  # Last smoothed value
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = next_value.round(2)


        ##################################################################

        last_season = df_defense["season_defense"].iloc[-1] 
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM','PACE', 'DefRtg', 'OffRtg']]

        # display(df_for_schedule)


        first_team = schedule_df['schedule_team'].iloc[0]
        # print(first_team)
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        # print(schedule_team_result)

        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]

        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]

        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # print("dfrtg:",schedule_defrtg)
        # print("pace",schedule_pace)
        # print("offrtg",schedule_offrtg)

        # This is to turn the first date in schedule into seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9

        # This is to turn the home and away games into a 1 or a zero -> away is 1 and anything is zero
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)


        

        # display(schedule_df)





        X_future = pd.DataFrame({
        # 'EWMA_FGA_2': [next_value], 
        'PACE':[schedule_pace], 
        'team_pace':[df['team_pace'].iloc[-1]], 
        'USG':[df['USG'].iloc[-1]],
        'DefRtg':[schedule_defrtg],
        'MIN_x':[next_value], 
        'home_away':[schedule_df['home_away'].iloc[0]],
        'Date_in_Seconds':[schedule_df['Date_in_Seconds'].iloc[0]],
        'OffRtg':[df['OffRtg'].iloc[-1]],
        'team_offrtg': [schedule_offrtg]
        })

        display(X_future)

        future_predictions = model.predict(X_future)



        # print("this is future",future_predictions)

        fga_prediction_results.update({player:future_predictions[0]})

        
    



    return fga_prediction_results

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display


def his_usage_team(player_names: dict, date_list: list, usage_path,player_base_path,defense_base_path):
    current_player_dic = {}

    for player, team in player_names.items():
        current_player_frames =[]

        for date in date_list:
            usage_path =build_data_path(usage_path,date=date)
            usage_data = pd.read_csv(usage_path)

            #merging player and defense dat into one
            merged_data, current_defense_df = his_player_defense_data(player_base_path,defense_base_path,player,date)

            #adding season to usage_data
            usage_data['season'] = date

            #Getting the player usage percentage for usage data and adding to merge
            player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
            merged_data['USG'] = player_usage

            #adding the current player team pace
            team_stat = current_defense_df.loc[current_defense_df['TEAM'] == team, 'PACE'].values[0]
            merged_data["team_pace"] = team_stat

            # adding current player team OffRtg
            team_offrtg = current_defense_df.loc[current_defense_df['TEAM'] == team, 'OffRtg'].values[0]
            merged_data["team_offrtg"] = team_offrtg

            team_poss = current_defense_df.loc[current_defense_df['TEAM'] == team, 'POSS'].values[0]
            merged_data["team_poss"] = team_poss
            
            # Exclude rows where the TEAM column matches the given team
            merged_data = merged_data[merged_data['TEAM'] != team]


            # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

            # Turn date into seconds
            merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
            merged_data = merged_data.sort_values(by="Date_in_Seconds")


            # Turn Home/Away game into 1 and 0
            merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
            # Dropping duplicates
            merged_data = merged_data.drop_duplicates()
            
            # Append the DataFrame for this date to the player's list
            current_player_frames.append(merged_data)

        # Combine all dates for the current player into one DataFrame
        current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)


    return current_player_dic, current_defense_df


player_names = { 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC', 'Shai Gilgeous-Alexander':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
usage_path = "D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"


def fga_prediction(player_names: dict, date_list: list, usage_path, player_base_path, defense_base_path, schedule_base_path):
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, usage_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    from sklearn.preprocessing import StandardScaler




    for player, team in player_names.items():
        # Get schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)
        
        # Get player-specific prediction data

        df = fga_prediction_data[player]





        
        # Model training and prediction code
        features = ['PACE', 'team_pace', 'USG', 'DefRtg', 'MIN_x', 'home_away', 'Date_in_Seconds', 'OffRtg', 'team_offrtg']
        target = 'FGA'

        timestamp = int(pd.Timestamp('2024-12-31').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        X_train = train_data[features]
        y_train = train_data[target]
        X_test = test_data[features]
        y_test = test_data[target]

        # Initialize Scaler
        scaler = StandardScaler()
        # Transform Data
        scaled_data_x = scaler.fit_transform(X_train)
        scaled_data_y = scaler.fit_transform(y_train.values.reshape(-1, 1))

        X_train = pd.DataFrame(scaled_data_x, columns=X_train.columns)     
        y_train = pd.DataFrame(scaled_data_y, columns=['features'])   

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # EWMA calculation for minutes
        alpha = 0.2
        df['EWMA_MIN'] = df['MIN_x'].ewm(span=(2/alpha - 1), adjust=False).mean()
        last_actual = df['MIN_x'].iloc[-1]
        last_smoothed = df['EWMA_MIN'].iloc[-1]
        next_value = alpha * last_actual + (1 - alpha) * last_smoothed
        next_value = round(next_value, 2)

        # Get defensive stats for the scheduled team
        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, ['TEAM', 'PACE', 'DefRtg', 'OffRtg']]

        first_team = schedule_df['schedule_team'].iloc[0]
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        schedule_defrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'DefRtg'].values[0]
        schedule_pace = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'PACE'].values[0]
        schedule_offrtg = df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, 'OffRtg'].values[0]

        # Convert schedule dates to seconds
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # Create future prediction dataframe
        X_future = pd.DataFrame({
            'PACE': [schedule_pace],
            'team_pace': [df['team_pace'].iloc[-1]],
            'USG': [df['USG'].iloc[-1]],
            'DefRtg': [schedule_defrtg],
            'MIN_x': [next_value],
            'home_away': [schedule_df['home_away'].iloc[0]],
            'Date_in_Seconds': [schedule_df['Date_in_Seconds'].iloc[0]],
            'OffRtg': [df['OffRtg'].iloc[-1]],
            'team_offrtg': [schedule_offrtg]
        })

        future_predictions = model.predict(X_future)
        fga_prediction_results[player] = future_predictions[0]

    return fga_prediction_results

results = fga_prediction(player_names, date_list, usage_path, player_base_path, defense_base_path, schedule_base_path)


for player, fga_predictions in results.items():
    print(fga_predictions)
   

            

            

            

            



            


In [ ]:
import data_functions as data_functions
import pandas as pd
import numpy as np
import importlib
importlib.invalidate_caches()
from data_functions import his_player_defense_data, current_player_defense_data

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"



current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # display(current_player_defense)

    # adding current player team OffRtg
    team_offrtg = current_player_defense.loc[current_player_defense['TEAM'] == team, 'OffRtg'].values[0]
    merged_data["team_offrtg"] = team_offrtg

    team_poss = current_player_defense.loc[current_player_defense['TEAM']== team, 'POSS'].values[0]
    merged_data["team_poss"] = team_poss

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    # display(merged_data.head(5))

    # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)



specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():

    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()



    alpha = 0.2

    df['EWMA_FGA'] = df['FGA'].ewm(span=(2/(1-alpha)-1), adjust=False).mean()

    df['EWMA_FGA_2'] = df['FGA'].ewm(span=(2/alpha - 1), adjust=False).mean()

    # alpha = 0.2  # Example smoothing factor
    df['Exp_smooth'] = 21  # Initialize column

    for i in range(1, len(df)):
        df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']

    df_act = df

    # display(df_act)

    # display(df)
    df = df.head(149)

    value_fga_list = []
    moving_average_list = []
    next_next_value_list = []
    date_list = []
    
    for value_fga, moving_average, date in zip(df['FGA'],df['EWMA_FGA_2'],df['Date']):
       value_fga_list.append(value_fga)
       moving_average_list.append(moving_average)
       #print("this is actual:",value_fga, "this is last_predicted",moving_average)
       next_next_value = alpha * value_fga + (1 - alpha) * moving_average
       next_next_value_list.append(next_next_value)
       date_list.append(date)
       #print("this is next:",next_next_value)
       dataframe_dic= {'Date':date_list,'Actual_FGA': value_fga_list, 'Moving_average':moving_average_list, 'Next':next_next_value_list}
       dataframe = pd.DataFrame(dataframe_dic)
      #  dataframe['Next'] = dataframe['Next'].shift(1)

    # display(dataframe)


    # Predict the next (11th) value
    last_actual = df['FGA'].iloc[-1]  # Last known FGA
    last_smoothed = df['EWMA_FGA_2'].iloc[-1]  # Last smoothed value
    next_value = alpha * last_actual + (1 - alpha) * last_smoothed


    # Print and add the prediction
    print(f"Predicted value: {next_value}")

    # display(df)

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error



    features = ['PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
    target = 'FGA'


     # Continue with your existing operations
    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]


    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    # print(y_pred)

     # Compare predictions with actual points
    predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  
        "Predicted Points": y_pred.round(1), 
        "Actual Points": y_test}).reset_index(drop=True)
    
    display(predicted_vs_actual)

    X_future = pd.DataFrame({
    # 'EWMA_FGA_2': [next_value], 
    'PACE':[df['PACE'].iloc[-1]], 
    'team_pace':[df['team_pace'].iloc[-1]], 
    'USG':[df['USG'].iloc[-1]],
    'DefRtg':[df['DefRtg'].iloc[-1]],
    'MIN_x':[30.09], 
    'home_away':[1],
    'Date_in_Seconds':[1738368000],
    'OffRtg':[df['OffRtg'].iloc[-1]],
    'team_offrtg': [df['team_offrtg'].iloc[-1]]
    })
    
    future_predictions = model.predict(X_future)

    print("this is future",future_predictions)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}, RMSE: {rmse}")

    import numpy as np

    # Drop NaN values (first row of 'Next' is NaN due to shift)
    rmse_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute RMSE
    rmse = np.sqrt(((rmse_df['Actual_FGA'] - rmse_df['Next']) ** 2).mean())

    print("RMSE:", rmse)


    # Drop NaN values (due to shift)
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Total number of valid comparisons
    total = len(comparison_df)

    # Count occurrences
    higher_count = (comparison_df['Next'] > comparison_df['Actual_FGA']).sum()
    lower_count = (comparison_df['Next'] < comparison_df['Actual_FGA']).sum()
    equal_count = (comparison_df['Next'] == comparison_df['Actual_FGA']).sum()

    # Calculate percentages
    higher_percent = (higher_count / total) * 100
    lower_percent = (lower_count / total) * 100
    equal_percent = (equal_count / total) * 100

    # Print results
    print(f"Percentage of predictions HIGHER than actual: {higher_percent:.2f}%")
    print(f"Percentage of predictions LOWER than actual: {lower_percent:.2f}%")
    print(f"Percentage of predictions EQUAL to actual: {equal_percent:.2f}%")


    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Filter cases where Next is higher than Actual
    higher_cases = comparison_df[comparison_df['Next'] > comparison_df['Actual_FGA']]
    higher_difference_avg = (higher_cases['Next'] - higher_cases['Actual_FGA']).mean()

    # Filter cases where Next is lower than Actual
    lower_cases = comparison_df[comparison_df['Next'] < comparison_df['Actual_FGA']]
    lower_difference_avg = (lower_cases['Actual_FGA'] - lower_cases['Next']).mean()

    # Print results
    print(f"Average difference when prediction is HIGHER: {higher_difference_avg:.2f}")
    print(f"Average difference when prediction is LOWER: {lower_difference_avg:.2f}")

    # Drop NaN values
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute Mean Error (ME)
    mean_error = (comparison_df['Next'] - comparison_df['Actual_FGA']).mean()

    # Print result
    print(f"Mean Error (ME): {mean_error:.2f}")

    